In [37]:
import pandas as pd 
import re 

# Loading in the data
This code takes the csv file from MZmine3 output 

In [38]:
df = pd.read_csv(r"C:\Users\adamg\OneDrive - The University of Liverpool\Project Work\MZmine\New_dataset_03Jul24.csv")
columns_to_drop = ['manual_annotation:identity', 'manual_annotation:comment', 'manual_annotation:compound_name','manual_annotation:ion_adduct','manual_annotation:mol_formula','manual_annotation:inchi','manual_annotation:smiles']

df.drop(columns_to_drop, inplace=True, axis=1)
df.head()

,id,area,rt,mz_range:min,mz_range:max,charge,fragment_scans,alignment_scores:rate,alignment_scores:aligned_features_n,alignment_scores:align_extra_features,...,datafile:batch06_C04_rep03_69.mzML:rt_range:max,datafile:batch06_C04_rep03_69.mzML:feature_state,datafile:batch06_C04_rep03_69.mzML:mz,datafile:batch06_C04_rep03_69.mzML:intensity_range:min,datafile:batch06_C04_rep03_69.mzML:intensity_range:max,datafile:batch06_C04_rep03_69.mzML:height,datafile:batch06_C04_rep03_69.mzML:charge,datafile:batch06_C04_rep03_69.mzML:isotopes,datafile:batch06_C04_rep03_69.mzML:asymmetry_factor,datafile:batch06_C04_rep03_69.mzML:tailing_factor
0,1,1561.0,0.5972,132.97134,132.97200,NaN,0,0.007,1,0,...,0.6281,ESTIMATED,132.97198,13380.0,13380.0,13380.0,NaN,NaN,1.8978,1.4489
1,2,17010.0,0.6124,101.77150,101.77201,NaN,0,0.007,1,184,...,0.7833,ESTIMATED,101.77205,10470.0,17010.0,17010.0,NaN,NaN,0.8632,0.9316
2,3,128.5,0.5031,116.35756,116.35810,NaN,0,0.007,1,0,...,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,8343.0,0.6105,101.81728,101.81785,NaN,0,0.007,1,103,...,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,342.7,0.6072,74.04131,74.04152,NaN,0,0.007,1,3,...,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Taking a look at the data, it is important to get a feel for what the data looks like

In [39]:
# Shape of the dataframe
df.shape

(8879, 2486)

In [40]:
for col in df.columns:
    print(col)

id
area
rt
mz_range:min
mz_range:max
charge
fragment_scans
alignment_scores:rate
alignment_scores:aligned_features_n
alignment_scores:align_extra_features
alignment_scores:weighted_distance_score
alignment_scores:mz_diff_ppm
alignment_scores:mz_diff
alignment_scores:rt_absolute_error
alignment_scores:ion_mobility_absolute_error
rt_range:min
rt_range:max
mz
intensity_range:min
intensity_range:max
height
datafile:batch08_B09_rep02_02.mzML:area
datafile:batch08_B09_rep02_02.mzML:rt
datafile:batch08_B09_rep02_02.mzML:mz_range:min
datafile:batch08_B09_rep02_02.mzML:mz_range:max
datafile:batch08_B09_rep02_02.mzML:fragment_scans
datafile:batch08_B09_rep02_02.mzML:fwhm
datafile:batch08_B09_rep02_02.mzML:rt_range:min
datafile:batch08_B09_rep02_02.mzML:rt_range:max
datafile:batch08_B09_rep02_02.mzML:feature_state
datafile:batch08_B09_rep02_02.mzML:mz
datafile:batch08_B09_rep02_02.mzML:intensity_range:min
datafile:batch08_B09_rep02_02.mzML:intensity_range:max
datafile:batch08_B09_rep02_02.mzML:he

# Only taking the necessary columns
For now I only need the id and then the columns that contain the intensities 

In [55]:
first_pattern = r"(.*?).mzML:mz$"
filtered_cols = [col for col in df.columns if re.search(first_pattern, col)]
test_df = df[filtered_cols]
test_df.shape

(8879, 145)

In [53]:
# Getting the intensities 
pattern = r"datafile:batch(\d+)_(S|B|C|QC)(\d+)_rep(\d+)_(\d+)\.mzML:mz$"


filtered_cols = [col for col in test_df.columns if re.search(pattern, col)]
unmatched_cols = [col for col in test_df.columns if col not in filtered_cols]

print([col for col in unmatched_cols])

filtered_df = pd.concat([df[['id']], test_df[filtered_cols]], axis=1)
filtered_df.shape

[]


(8879, 146)

In [56]:
# Function to rename the columns to standardise for the PCA 

def rename_matching_columns(df, pattern):
    def replace_func(matchobj):
        batch_num = matchobj.group(1)
        qc_type = matchobj.group(2)  # This will capture "S", "B", or "QC"
        qc_num = matchobj.group(3)
        rep_num = matchobj.group(4)

        return f"batch{batch_num}_{qc_type}{qc_num}_rep{rep_num}"
    
    cols = [col for col in df.columns if re.search(pattern, col)]
    renamed_cols = [re.sub(pattern, replace_func, col) for col in cols]
    return df.rename(columns=dict(zip(cols, renamed_cols)))

filtered_df = rename_matching_columns(filtered_df,pattern)

filtered_df.to_csv("MZmine_Feature_Matrix.csv", index=False)
filtered_df



,id,batch08_B09_rep02,batch07_B07_rep01,batch07_B07_rep02,batch08_B09_rep01,batch07_B07_rep03,batch06_B05_rep03,batch07_QC30_rep01,batch06_B05_rep01,batch06_B05_rep02,...,batch07_S05_rep02,batch05_C03_rep02,batch07_C03_rep03,batch06_C03_rep03,batch01_QC04_rep02,batch03_QC12_rep01,batch05_S05_rep01,batch06_C03_rep02,batch01_QC04_rep03,batch06_C04_rep03
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,132.97198,132.97197,NaN,132.97199,NaN,NaN,132.97198
1,2,101.77184,101.77213,101.7718,101.77162,101.77186,101.77174,101.77161,101.77160,101.77132,...,101.77171,101.77195,101.77173,101.77195,101.77167,101.77186,101.77156,101.77180,101.77184,101.77205
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,101.81774,NaN,101.81773,101.81747,...,NaN,101.81756,NaN,NaN,101.81760,NaN,101.81755,101.81785,101.81782,NaN
4,5,NaN,NaN,NaN,NaN,NaN,74.04131,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8874,8875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,538.17113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8875,8876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,579.54413,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8876,8877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8877,8878,NaN,NaN,NaN,NaN,NaN,NaN,553.35293,553.35184,NaN,...,553.35233,553.35182,553.35320,NaN,NaN,553.35165,NaN,553.35148,553.35131,553.35185
